# Modeling

## Carga de datos

In [1]:
import math
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import shuffle
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report, accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, VotingClassifier, GradientBoostingClassifier
from imblearn.over_sampling import SMOTE

import sys
sys.path.append('../src')

import funciones as fn
import utils as consts

In [111]:
# Load the dataset
df_curated_stocks = pd.read_csv('../data/clean/df_clean_stocks.csv')

In [112]:
df_curated_stocks.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872696 entries, 0 to 1872695
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Date       1872696 non-null  object 
 1   Symbol     1872696 non-null  object 
 2   Adj Close  1872696 non-null  float64
 3   Close      1872696 non-null  float64
 4   High       1872696 non-null  float64
 5   Low        1872696 non-null  float64
 6   Open       1872696 non-null  float64
 7   Volume     1872696 non-null  int64  
 8   Year       1872696 non-null  int64  
 9   Month      1872696 non-null  int64  
 10  Day        1872696 non-null  int64  
dtypes: float64(5), int64(4), object(2)
memory usage: 157.2+ MB


In [113]:
# Variables derivada: cambio entre high y low
df_curated_stocks['Cambio_CloseOpen'] = df_curated_stocks['Close'] - df_curated_stocks['Open']
df_curated_stocks['Accion_Sube'] = np.where(df_curated_stocks['Cambio_CloseOpen'] > 0, 1, 0)

# Drop unsed variables
df_curated_stocks.drop(columns=['High', 'Low'], inplace=True)
df_curated_stocks.drop(columns=['Adj Close', 'Volume'], inplace=True)
df_curated_stocks.drop(columns=['Year', 'Month', 'Day'], inplace=True)


In [114]:
# Creaccion de variables lag
lag_columns = ['Open', 'Close', 'Cambio_CloseOpen', 'Accion_Sube']
df_curated_stocks = fn.create_lag_variables(df_curated_stocks, lag_columns, consts.NUM_VARIABLES_LAG)


In [116]:
df_curated_stocks.tail(5)

,Date,Symbol,Close,Open,Cambio_CloseOpen,Accion_Sube,Open_Lag1,Close_Lag1,Cambio_CloseOpen_Lag1,Accion_Sube_Lag1,...,Cambio_CloseOpen_Lag3,Accion_Sube_Lag3,Open_Lag4,Close_Lag4,Cambio_CloseOpen_Lag4,Accion_Sube_Lag4,Open_Lag5,Close_Lag5,Cambio_CloseOpen_Lag5,Accion_Sube_Lag5
1872691,2024-12-16,XYL,120.779999,120.720001,0.059998,1,120.930000,120.599998,-0.330002,0.0,...,-5.989998,0.0,129.910004,128.570007,-1.339996,0.0,127.589996,129.350006,1.760010,1.0
1872692,2024-12-17,XYL,120.769997,119.730003,1.039993,1,120.720001,120.779999,0.059998,1.0,...,-1.010002,0.0,129.369995,123.379997,-5.989998,0.0,129.910004,128.570007,-1.339996,0.0
1872693,2024-12-18,XYL,116.919998,120.790001,-3.870003,0,119.730003,120.769997,1.039993,1.0,...,-0.330002,0.0,122.080002,121.070000,-1.010002,0.0,129.369995,123.379997,-5.989998,0.0
1872694,2024-12-19,XYL,116.430000,117.440002,-1.010002,0,120.790001,116.919998,-3.870003,0.0,...,0.059998,1.0,120.930000,120.599998,-0.330002,0.0,122.080002,121.070000,-1.010002,0.0
1872695,2024-12-20,XYL,117.139999,116.070000,1.070000,1,117.440002,116.430000,-1.010002,0.0,...,1.039993,1.0,120.720001,120.779999,0.059998,1.0,120.930000,120.599998,-0.330002,0.0


In [22]:
# Set index to Date
df_curated_stocks['Date'] = pd.to_datetime(df_curated_stocks['Date'])
df_curated_stocks.set_index('Date', inplace=True)


In [23]:
# Quitar nans
df_curated_stocks.dropna(inplace=True)

# Verificar
columns_with_nans = df_curated_stocks.columns[df_curated_stocks.isna().any()]
print("Columnas con NaNs: ", columns_with_nans)


Columnas con NaNs:  Index([], dtype='object')


## Clasificacion

La predicción de si una acción sube o baja es un problema de clasificación, ya que el objetivo es asignar una etiqueta discreta (por ejemplo, 1 si sube y 0 si baja). Por lo que se probaran algunos modelos para analizar que tan bien se ajustan al probelma. Por ejemplo se probaran:  
- Regresión Logística, 
- Árboles de Decisión 
- Random Forest 
- Ensambladores de Boosting 

In [24]:
# Separar características y objetivo
x = df_curated_stocks.drop(['Accion_Sube', 'Symbol', 'Close', 'Open', 'Cambio_CloseOpen'], axis=1)
y = df_curated_stocks['Accion_Sube']

# Rolling mean / media mobil / dias de lag 


In [25]:
# Dataset 
x.head()

,Open_Lag1,Close_Lag1,Cambio_CloseOpen_Lag1,Accion_Sube_Lag1,Open_Lag2,Close_Lag2,Cambio_CloseOpen_Lag2,Accion_Sube_Lag2,Open_Lag3,Close_Lag3,Cambio_CloseOpen_Lag3,Accion_Sube_Lag3,Open_Lag4,Close_Lag4,Cambio_CloseOpen_Lag4,Accion_Sube_Lag4
Date,,,,,,,,,,,,,,,,
2010-01-08,7.356667,7.365000,0.008333,1.0,7.335000,7.343333,0.008333,1.0,7.431667,7.340000,-0.091667,0.0,7.295000,7.435000,0.140000,1.0
2010-01-11,7.331667,7.473333,0.141666,1.0,7.356667,7.365000,0.008333,1.0,7.335000,7.343333,0.008333,1.0,7.431667,7.340000,-0.091667,0.0
2010-01-12,7.491667,7.438333,-0.053334,0.0,7.331667,7.473333,0.141666,1.0,7.356667,7.365000,0.008333,1.0,7.335000,7.343333,0.008333,1.0
2010-01-13,7.393333,7.386667,-0.006666,0.0,7.491667,7.438333,-0.053334,0.0,7.331667,7.473333,0.141666,1.0,7.356667,7.365000,0.008333,1.0
2010-01-14,7.405000,7.345000,-0.060000,0.0,7.393333,7.386667,-0.006666,0.0,7.491667,7.438333,-0.053334,0.0,7.331667,7.473333,0.141666,1.0


In [26]:
# Columna que quiero predecir
y.head(10)

Date
2010-01-08    1
2010-01-11    0
2010-01-12    0
2010-01-13    0
2010-01-14    1
2010-01-15    0
2010-01-19    1
2010-01-20    1
2010-01-21    0
2010-01-22    0
Name: Accion_Sube, dtype: int64

In [ ]:
# Dividir en conjunto de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=12345, shuffle=False)


Antes de realizar el analisis de los modelos, debemos determinar si las clases que queremos predecri (sube o baja la accion), esta desbalanceado. Este analis se lo debe realizar sobre los datos de entrenamiento que queremos predecir ```y_train```

In [28]:
# Verificar el balance de los datos de entrenamiento
print("Distribución de clases antes del sobremuestreo:")
print(y_train.value_counts())

# Calcular el porcentaje de cada clase
total = y_train.value_counts().sum()
porcentajes = (y_train.value_counts() / total) * 100

print("\nPorcentaje de cada clase:")
print(porcentajes.round(2))


Distribución de clases antes del sobremuestreo:
Accion_Sube
0    1083229
1     414924
Name: count, dtype: int64

Porcentaje de cada clase:
Accion_Sube
0    72.3
1    27.7
Name: count, dtype: float64


Segun los resultados obtenidos, se puede observar que las clases estan desbalanceadas. 
- El 72.3% son acciones que bajan
- El 27.7% son acciones que suben

Debido a que la prediccion que queremos realizar es sobre el mercado de valores, el numero real de acciones que suben es relevante (asi este desbalanceado). Por esto se puede realizar el analis sin balancear el dataset. Sin embargo, dejo el detalle de los posibles metodos a realizar si se considera el balanceo.

Si se considera un sobremuestreo debido a que la diferencia entre clases es considerable, se puede usar una tasa de crecimiento o la técnica SMOTE (Synthetic Minority Oversampling Technique). Este segundo metodo genera ejemplos sintéticos para la clase minoritaria, evitando el sobreajuste (overfitting).

Este metodo realiza lo siguiente:
1. Identifica de la clase minoritaria
2. Crea ejemplos sintéticos: para cada muestra de la clase minoritaria, encuentra sus k_neighbors (vecinos más cercanos), genera nuevos ejemplos interpolando entre la muestra actual y uno de sus vecinos seleccionados al azar.
3. Resultado: Se obtiene un dataset balanceado con más ejemplos de la clase minoritaria

In [138]:
# Aplicar SMOTE al conjunto de entrenamiento
#smote = SMOTE(random_state=12345)
#x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)


In [ ]:
# Analizar tasa de crecimiento
#print((1083229 / 414924).__round__(3))  

# Porcentaje de acciones que suben con respecto a las que bajan
#print((414924 * 2 / 1083229 * 100 ).__round__(2))

2.611
76.61


In [ ]:
# Sobremuestreo manual con tasa de crecimiento
#x_train_resampled, y_train_resampled = fn.sobremuestreo(x_train, y_train, 2)

In [ ]:
# Verificar el balance después del sobremuestreo
#print("\nDistribución de clases después del sobremuestreo:")
#print(y_train_resampled.value_counts())
#
## Calcular el porcentaje de cada clase
#total = y_train_resampled.value_counts().sum()
#porcentajes = (y_train_resampled.value_counts() / total) * 100
#
#print("\nPorcentaje de cada clase:")
#print(porcentajes.round(2))



Distribución de clases después del sobremuestreo:
Accion_Sube
0    1083229
1     829848
Name: count, dtype: int64

Porcentaje de cada clase:
Accion_Sube
0    56.62
1    43.38
Name: count, dtype: float64


Una vez realizado el sobremuestreo, se puede proceder a entrenar el modelo. En este caso, se utilizará un modelo de regresión logística y un árbol de decisión. Se compararán los resultados de ambos modelos para determinar cuál es el más efectivo en la predicción de acciones que suben o bajan.

### Regresion Logistica

El primer modelo que se va a probar es la regresion logitsica, la cual es un modelo de clasificación supervisado que se utiliza para predecir la probabilidad de que una instancia pertenezca a una clase binaria; en este caso si sube (1) o baja (0) la accion.

In [29]:
# Modelo
modelo_reg_log = LogisticRegression(random_state=12345, max_iter=1000)

# Entrenar el modelo 
#modelo_reg_log.fit(x_train_resampled, y_train_resampled)
modelo_reg_log.fit(x_train, y_train)

# Predicciones en el conjunto de prueba
y_pred = modelo_reg_log.predict(x_test)

Una vez entrenado el modelo, continuamos con el reporte de metricas para analizar que tan bien realizo las predicciones.

In [30]:
# Evaluar el modelo datos de validacion/prueba
fn.reporte(y_test, y_pred)


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90    308894
           1       0.54      0.48      0.51     65645

    accuracy                           0.84    374539
   macro avg       0.72      0.70      0.71    374539
weighted avg       0.83      0.84      0.83    374539

Exactitud:  0.8374188001783526
Recall: 0.48221494401706144
Precision: 0.5405751562553366

RMSE:  0.4032135908196144
R^2:  -0.12474298255856198


In [31]:
# Predicciones 
fn.predicciones(y_pred)


Conteo de clases:
0    315981
1     58558
Name: count, dtype: int64

Total de datos sobre los que predice:  374539
Datos que fueron predichos:  58558
El modelo predice que el 15.63 % de acciones suben


**Precision**

El modelo tiene una precision de 0.57 para los datos de prueba

**Clasificacion:**

- Sube la accion: 1
- Baja la accion: 0

Buenos: VP - VN

Malos: FP - FN

VP: El modelo predice que sube la accion -> y en la realidad sube la accion

VN: El modelo predice que baja la aacion -> y en la realidad baja la accion

FP: El modelo predice que sube la accion -> y en la realidad baja la accion

FN: El modelo predice que baja la accion -> y en la realidad sube la accion

### Arboles de decision

In [32]:
# Modelo
arbol_clf = DecisionTreeClassifier(max_depth=3, max_leaf_nodes=2, random_state=12345)

# Entrenar el modelo 
#arbol_clf.fit(x_train_resampled, y_train_resampled)
arbol_clf.fit(x_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_arbol = arbol_clf.predict(x_test)


In [35]:
# Evaluar el modelo datos de validacion/prueba
fn.reporte(y_test, y_pred_arbol)


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.82      0.90    308894
           1       0.53      0.95      0.68     65645

    accuracy                           0.84    374539
   macro avg       0.76      0.89      0.79    374539
weighted avg       0.91      0.84      0.86    374539

Exactitud:  0.8421072304886807
Recall: 0.9541320740345799
Precision: 0.5273997979117548

RMSE:  0.3973572316081831
R^2:  -0.0923082416626817


In [36]:
# Predicciones 
fn.predicciones(y_pred_arbol)


Conteo de clases:
0    255779
1    118760
Name: count, dtype: int64

Total de datos sobre los que predice:  374539
Datos que fueron predichos:  118760
El modelo predice que el 31.71 % de acciones suben


### Random Forest

In [37]:
# Modelo
bosque_aleatorio_clf = RandomForestClassifier(random_state=12345, n_estimators=500, n_jobs=-1)

# Entrenar el modelo 
#bosque_aleatorio_clf.fit(x_train_resampled, y_train_resampled)
bosque_aleatorio_clf.fit(x_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = bosque_aleatorio_clf.predict(x_test)

In [42]:
# Evaluar el modelo datos de validacion/prueba
fn.reporte(y_test, y_pred_rf)


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90    308894
           1       0.54      0.59      0.56     65645

    accuracy                           0.84    374539
   macro avg       0.73      0.74      0.73    374539
weighted avg       0.85      0.84      0.84    374539

Exactitud:  0.8405640000106798
Recall: 0.5877827709650393
Precision: 0.5416198764738911

RMSE:  0.39929437760795006
R^2:  -0.10298436936075617


In [43]:
# Predicciones 
fn.predicciones(y_pred_rf)


Conteo de clases:
0    303299
1     71240
Name: count, dtype: int64

Total de datos sobre los que predice:  374539
Datos que fueron predichos:  71240
El modelo predice que el 19.02 % de acciones suben


### Voting Classifier

In [40]:
modelo_voto_clf = VotingClassifier(
    estimators=[
        ('regresion_logistica', LogisticRegression(random_state=12345, max_iter=1000)),
        ('decision_tree', DecisionTreeClassifier(random_state=12345, max_depth=5)),
        ('random_forest', RandomForestClassifier(random_state=12345, max_depth=1, n_estimators=500, n_jobs=-1)),
    ],
    voting='hard',
    n_jobs=-1
)

# Entrenar el modelo 
#modelo_voto_clf.fit(x_train_resampled, y_train_resampled)
modelo_voto_clf.fit(x_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_voting = modelo_voto_clf.predict(x_test)


In [46]:
# Evaluar el modelo datos de validacion/prueba
fn.reporte(y_test, y_pred_voting)


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.97      0.83      0.90    308894
           1       0.53      0.88      0.66     65645

    accuracy                           0.84    374539
   macro avg       0.75      0.86      0.78    374539
weighted avg       0.89      0.84      0.86    374539

Exactitud:  0.8414424132066354
Recall: 0.8812247695940285
Precision: 0.5285963613768653

RMSE:  0.398192901485404
R^2:  -0.09690747314507031


In [47]:
# Predicciones 
fn.predicciones(y_pred_voting)


Conteo de clases:
0    265102
1    109437
Name: count, dtype: int64

Total de datos sobre los que predice:  374539
Datos que fueron predichos:  109437
El modelo predice que el 29.22 % de acciones suben


### Regression Boosting

In [60]:
# Modelo
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=12345)

# Entrenar el modelo
gbr.fit(x_train, y_train)

# Predicciones en el conjunto de prueba
y_pred_gbr = gbr.predict(x_test)

In [65]:
# Evaluar el modelo datos de validacion/prueba
# Calcular métricas
rmse_gbr = math.sqrt(mean_squared_error(y_test, y_pred_gbr))
r2_gbr = r2_score(y_test, y_pred_gbr)

# Imprimir resultados
print(f"Gradient Boosting Regressor - RMSE: {rmse_gbr}")
print(f"Gradient Boosting Regressor - R^2: {r2_gbr}")

Gradient Boosting Regressor - RMSE: 0.3050758136756918
Gradient Boosting Regressor - R^2: 0.39686154481492475


In [ ]:
# Predicciones 
fn.predicciones(y_pred_gbr)


Conteo de clases:
0    265102
1    109437
Name: count, dtype: int64

Total de datos sobre los que predice:  374539
Datos que fueron predichos:  109437
El modelo predice que el 29.22 % de acciones suben


## Model Testing

En esta seccion realizaremos pruebas de los modelos vistos anteriormete con diferentes parametros para determinar cual nos da mejores reultados. Para considerar esto, debemos tomar en cuenta el resultado de precision. Al ser un problema del mercado bursatil nos interesa analizar la presision de las acciones que suben que el modelo logra capturar realmente. Esto permitira a los interesados tomar decisiones al comprar o vender acciones con precision, con un menor riesgo de perdida.


Se realizara el analisis con el dataset sin sobremuestreo y otro con el dataset sobremuestreado.

### Data Preparation

In [40]:
consts.NUM_VARIABLES_LAG = 20
consts.TASA_CRECIMIENTO = 2

# Load the dataset
df_curated_stocks = pd.read_csv('../data/clean/df_clean_stocks.csv')

# Variables derivada: cambio entre high y low
df_curated_stocks['Cambio_CloseOpen'] = df_curated_stocks['Close'] - df_curated_stocks['Open']
df_curated_stocks['Accion_Sube'] = np.where(df_curated_stocks['Cambio_CloseOpen'] > 0, 1, 0)

# Drop unused variables
df_curated_stocks.drop(columns=['High', 'Low', 'Symbol', 'Adj Close', 'Volume', 'Year', 'Month', 'Day'], inplace=True)

# Creaccion de variables lag
lag_columns = ['Open', 'Close', 'Cambio_CloseOpen', 'Accion_Sube']
df_curated_stocks = fn.create_lag_variables(df_curated_stocks, lag_columns, consts.NUM_VARIABLES_LAG)

# Rolling mean / media mobil / dias de lag 




# Set index to Date
df_curated_stocks['Date'] = pd.to_datetime(df_curated_stocks['Date'])
df_curated_stocks.set_index('Date', inplace=True)

# Quitar nans
df_curated_stocks.dropna(inplace=True)
df_curated_stocks.head()

,Close,Open,Cambio_CloseOpen,Accion_Sube,Open_Lag1,Close_Lag1,Cambio_CloseOpen_Lag1,Accion_Sube_Lag1,Open_Lag2,Close_Lag2,...,Cambio_CloseOpen_Lag18,Accion_Sube_Lag18,Open_Lag19,Close_Lag19,Cambio_CloseOpen_Lag19,Accion_Sube_Lag19,Open_Lag20,Close_Lag20,Cambio_CloseOpen_Lag20,Accion_Sube_Lag20
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-02,7.093333,7.083333,0.010000,1,7.093333,7.083333,-0.010000,0.0,7.211667,7.096667,...,0.008333,1.0,7.431667,7.340000,-0.091667,0.0,7.295000,7.435000,0.140000,1.0
2010-02-03,7.108333,7.050000,0.058333,1,7.083333,7.093333,0.010000,1.0,7.093333,7.083333,...,0.008333,1.0,7.335000,7.343333,0.008333,1.0,7.431667,7.340000,-0.091667,0.0
2010-02-04,6.910000,7.060000,-0.150000,0,7.050000,7.108333,0.058333,1.0,7.083333,7.093333,...,0.141666,1.0,7.356667,7.365000,0.008333,1.0,7.335000,7.343333,0.008333,1.0
2010-02-05,6.948333,6.913333,0.035000,1,7.060000,6.910000,-0.150000,0.0,7.050000,7.108333,...,-0.053334,0.0,7.331667,7.473333,0.141666,1.0,7.356667,7.365000,0.008333,1.0
2010-02-08,6.990000,6.993333,-0.003333,0,6.913333,6.948333,0.035000,1.0,7.060000,6.910000,...,-0.006666,0.0,7.491667,7.438333,-0.053334,0.0,7.331667,7.473333,0.141666,1.0


In [ ]:
# Separar características y objetivo
x = df_curated_stocks.drop(['Accion_Sube', 'Close', 'Open', 'Cambio_CloseOpen'], axis=1)
y = df_curated_stocks['Accion_Sube']

# Dividir en conjunto de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=12345, shuffle=False)

# Sobremuestreo manual con tasa de crecimiento
x_train_resampled, y_train_resampled = fn.sobremuestreo(x_train, y_train, consts.TASA_CRECIMIENTO)


### Modelos

In [42]:
# Resultados de los modelos
model_results = []

# Diccionario para almacenar el mejor modelo
best_model = {
    'name': None,
    'params': None,
    'Exactitud': float('-inf'),
    'Recall': float('-inf'),
    'Precision': float('-inf'),
    'F1': float('-inf'),
}

In [43]:
# Lista de modelos y sus hiperparámetros
models = [
    #{
    #    'name': 'LinearRegression',
    #    'model': LinearRegression(n_jobs=-1 ),
    #    'params': [{}]  # Sin hiperparámetros ajustables
    #},
    {
        'name': 'LogisticRegression',
        'model': LogisticRegression(n_jobs=-1, random_state=12345),
        'params': [
           {'max_iter': mx}
            for mx in [100, 500, 1000]
        ]  
    },
    {
        'name': 'DecisionTreeClassifier',
        'model': DecisionTreeClassifier(random_state=12345),
        'params': [
            {'max_depth': md, 'max_leaf_nodes': mln}
            for md in [3, 5, 10]
            for mln in [2, 3, 5]
            ]  
    },
    {
        'name': 'RandomForestClassifier',
        'model': RandomForestClassifier(n_jobs=-1, random_state=12345),
        'params': [
            {'n_estimators': n}
            for n in [100, 300, 500]
            ]  
    },
    {
        'name': 'VotingClassifier',
        'model': VotingClassifier(estimators=[
            ('regresion_logistica', LogisticRegression(random_state=12345, max_iter=1000)),
            ('decision_tree', DecisionTreeClassifier(random_state=12345, max_depth=5)),
            ('random_forest', RandomForestClassifier(random_state=12345, max_depth=1, n_estimators=500, n_jobs=-1)),
        ], 
            voting='hard',
            n_jobs=-1),
        'params': [{}]  # Sin hiperparámetros ajustables en este caso
    },
    #{
    #    'name': 'GradientBoostingRegressor',
    #    'model': GradientBoostingClassifier(random_state=12345),
    #    'params': [
    #        {'n_estimators': n, 'learning_rate': lr, 'max_depth': md}
    #        for n in [10, 50, 100]
    #        for lr in [0.1, 0.3, 0.05]
    #        for md in [3, 5, 10]
    #    ]
    #}
]

In [44]:
def buscar_mejor_modelo(x_train, y_train):
    # Iterar sobre los modelos y sus hiperparámetros
    for model_info in models:
        for params in model_info['params']:
            model = model_info['model'].set_params(**params)
            model.fit(x_train, y_train)
            pred_test = model.predict(x_test)


            # Calcular métricas
            accuracy = accuracy_score(y_test, pred_test)
            recall = recall_score(y_test, pred_test)
            precision = precision_score(y_test, pred_test)
            f1 = f1_score(y_test, pred_test)

            # Guardar la información del modelo probado
            model_results.append({
                'name': model_info['name'],
                'params': params,
                'Exactitud': accuracy,
                'Recall': recall,
                'Precision': precision,
                'F1': f1,
            })
            
            # Actualizar el mejor modelo si es necesario
            if precision > best_model['Precision']:
                best_model['name'] = model_info['name']
                best_model['params'] = params
                best_model['Exactitud'] = accuracy
                best_model['Recall'] = recall
                best_model['Precision'] = precision
                best_model['F1'] = f1


    # Imprimir el mejor modelo
    print("Mejor modelo:", best_model['name'])
    print("Hiperparámetros:", best_model['params'])
    print("Exactitud:", best_model['Exactitud'])
    print("Recall:", best_model['Recall'])
    print("Precision:", best_model['Precision'])
    print("F1:", best_model['F1'])


In [26]:
# Sin modificacion
buscar_mejor_modelo(x_train, y_train)

pretty_json = json.dumps(model_results, indent=4)
print()
print(pretty_json)

Mejor modelo: RandomForestClassifier
Hiperparámetros: {'n_estimators': 100}
Exactitud: 0.8280535447669538
Recall: 0.5783936993759873
Precision: 0.5463569501031801
F1: 0.5619190665883019

[
    {
        "name": "LogisticRegression",
        "params": {
            "max_iter": 100
        },
        "Exactitud": 0.8240058611313436,
        "Recall": 0.7393822610098475,
        "Precision": 0.5274354671142012,
        "F1": 0.6156788298016241
    },
    {
        "name": "LogisticRegression",
        "params": {
            "max_iter": 500
        },
        "Exactitud": 0.8227029128370311,
        "Recall": 0.5657341952252384,
        "Precision": 0.5330166772218704,
        "F1": 0.5488883212591236
    },
    {
        "name": "LogisticRegression",
        "params": {
            "max_iter": 1000
        },
        "Exactitud": 0.822555530161117,
        "Recall": 0.5284838843391851,
        "Precision": 0.5350900078267675,
        "F1": 0.5317664299402548
    },
    {
        "name": 

In [45]:
# Con sobremuestreo
buscar_mejor_modelo(x_train_resampled, y_train_resampled)

pretty_json = json.dumps(model_results, indent=4)
print()
print(pretty_json)

Mejor modelo: RandomForestClassifier
Hiperparámetros: {'n_estimators': 100}
Exactitud: 0.8292667818672317
Recall: 0.8918900751728078
Precision: 0.531118864790217
F1: 0.6657718457562909

[
    {
        "name": "LogisticRegression",
        "params": {
            "max_iter": 100
        },
        "Exactitud": 0.8258748443403985,
        "Recall": 0.9908022540639249,
        "Precision": 0.5228835454862568,
        "F1": 0.6845201238390093
    },
    {
        "name": "LogisticRegression",
        "params": {
            "max_iter": 500
        },
        "Exactitud": 0.8260051391698296,
        "Recall": 0.9896371315580152,
        "Precision": 0.5231006099366377,
        "F1": 0.6844276743204032
    },
    {
        "name": "LogisticRegression",
        "params": {
            "max_iter": 1000
        },
        "Exactitud": 0.8259923232849676,
        "Recall": 0.9870828245258287,
        "Precision": 0.5231445196532478,
        "F1": 0.68385329147279
    },
    {
        "name": "D